### Environment Setup

In [ ]:
!pip install ipyvtklink

### Creating a scene
The scene-to-be-rendered is instantiated from a brain atlas within BrainGlobe's [Atlas API](https://gin.g-node.org/brainglobe/atlases). This is used to select a brain model. From there, you can select any number of brain regions to highlight in your render and how to color them.

In [ ]:
### sets up viewing window
from vedo import embedWindow
embedWindow(None)

from brainrender import Scene
from brainrender.actors import Points
from brainrender._colors import get_random_colors

In [ ]:
### instantiate the scene with allen mouse brain atlas and choose VISp to highlight
### to choose a different atlas, refer to these filenames; https://gin.g-node.org/brainglobe/atlases
scene = Scene(atlas_name="allen_mouse_25um", title='Probes')
scene.add_brain_region("VISp", alpha=0.15, color="green")

In [ ]:
### above, VISp is selected for coloring. Multiple region arguments can be supplied if needed.
### Region names and IDs can be viewed with each of these lines
# print(scene.atlas.lookup_df.head())
# print(scene.atlas.lookup_df)
# print(scene.atlas.hierarchy)

### Extract NWB CCF coordinates
Here you read the NWB file you're interested in viewing. Note that this will only work with ecephys NWB files which have a valid **electrodes** field.

In [ ]:
### choose the file you want to visualize here
nwb_filepath = "test.nwb"

In [ ]:
from pynwb import NWBHDF5IO

### read the nwb file
io = NWBHDF5IO(nwb_filepath, mode="r", load_namespaces=True)
nwb = io.read()

In [ ]:
import numpy as np

### read the x,y,z ccf coordinates and generate points
xs = nwb.electrodes.x
ys = nwb.electrodes.y
zs = nwb.electrodes.z
n = min(len(xs), len(ys), len(zs))
points = np.array([[xs[i], ys[i], zs[i]] for i in range(n)])

In [ ]:
### add the points to the scene all as one color
# scene.add(Points(points, name="units", colors="red"))

### add the points to the scene, coloring them based on CCF location label
for location in set(nwb.electrodes.location):
    these_points = np.array([points[i] for i in range(n) if nwb.electrodes.location[i] == location])
    scene.add(Points(these_points, colors=get_random_colors()))

### Rendering
Rendering is as simple as `scene.render()`.
This will create a pop-up window with the interactive 3D rendering of your scene. 
When you're done with it, press "Esc" to close the window!

In [ ]:
scene.render()